### Goal of Async IO is
#### 1. Maximize the usage of single thread
####    By Handling the io operations asynchronously and
####    By Enabling concurrent code using coroutines.

In [ ]:
import time
import threading
import asyncore


class Mayhem(threading.Thread):
    def __init__(self, map):
        super().__init__()
        self.map = map

    def run(self):
        for key, value in self.map.items():
            time.sleep(value)

In [ ]:
# concurrency Exception will be raised by worker thread

d = {"k1": 1, "k2": 2, "k3": 3}
m = Mayhem(d)
m.start()

d["k4"] = 4

In [ ]:
import asyncio

loop = asyncio.get_event_loop()

In [ ]:
import datetime


def print_now():
    print(datetime.datetime.now())


loop.call_soon(print_now)
loop.call_soon(print_now)

In [ ]:
import threading
import asyncio
import datetime


# Asyncion does not support startig event loop inside already running thread
def start_loop():
    def hog():
        sum = 0
        for i in range(10_000):
            for j in range(10_000):
                sum += i * j
        print(sum)

    new_loop = asyncio.new_event_loop()
    new_loop.set_debug(True)

    def trampolin(name=""):
        print(name, end="")
        print_now()
        # THESE CALLS WILL MAINTAIN THE ORDER OF SCHEDULING
        new_loop.call_later(1, trampolin)

    new_loop.call_soon(trampolin)
    new_loop.call_later(15, hog)
    new_loop.call_later(20, new_loop.stop)
    # EVENT Loop can be stop and rereun many time
    # remember putting long time taking function can block the event loop
    new_loop.run_forever()


thread = threading.Thread(target=start_loop)
thread.start()
thread.join()

In [ ]:
# UVLoop -> wrapper over libuv wrapper crated by yuri selivanov but originally created by nodejs

# USING COROUTINES

In [ ]:
import datetime
import asyncio
import threading


def print_now():
    print(datetime.datetime.now())


async def keep_printing(name):
    while True:
        print(name, end="")
        print_now()
        await asyncio.sleep(0.5)


def run_coroutine_in_thread(coro):
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    # raise TimeoutError after 10 sec
    # asyncio.wait_for(coro, 10)
    loop.run_until_complete(asyncio.wait_for(coro, 10))


thread = threading.Thread(target=run_coroutine_in_thread, args=(keep_printing("A"),))
thread.start()
print("Hello")

In [22]:
import inspect


async def async_printer(name="default"):
    print(name)
    try:
        await asyncio.sleep(1)
    except asyncio.CancelledError:
        print("Cancelled")

coroutine = await async_printer()

print(inspect.isawaitable(coroutine))
print(type(coroutine), type(async_printer))

default
False
<class 'NoneType'> <class 'function'>


In [23]:
import asyncio


async def async_main():
    await async_printer("First")
    await async_printer("Second")
    # WILL RUN THE TASKS ONE AFTER ANOTHER


async def async_main_cooperative_task():
    await asyncio.gather(
        async_printer("First"),
        async_printer("Second"),
        # WILL RUN THE TASKS IN CONTEXT SWITCH MANNER
    )


# run in main thread
loop = asyncio.get_event_loop()
# schedule the coroutine to run in event loop
loop.create_task(async_main_cooperative_task())

<Task pending name='Task-41' coro=<async_main_cooperative_task() running at /tmp/ipykernel_53953/1573941382.py:10>>

First
Second
